# Car Sales

In [1]:
import pandas as pd
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

## Get models

car sales from 2015/01 - 2016/11

In [7]:
url = 'https://xl.16888.com/style-201501-201611-1.html'

service = Service(r'D:\OneDrive - 东南大学\5 我的代码\crawling\chromedriver\chromedriver.exe')
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_experimental_option('excludeSwitches', ['enable-logging'])
prefs = {"profile.managed_default_content_settings.images": 2, 'permissions.default.stylesheet': 2}
chrome_options.add_experimental_option("prefs", prefs)
chrome_options.add_argument('lang=zh_CN.utf-8')
driver = webdriver.Chrome(service=service, options=chrome_options)

try:
    driver.get(url)
except:
    print("No Data.")
wait = WebDriverWait(driver, 10)

# define data file
data = []
columns = ['排名', '车型', '销量', '级别', '指导价(万元)', '燃油类型', '整备质量(kg)', '前轮制动', '后轮制动']
page = 1
# iterate pages
while True:

    # get items
    table = driver.find_element(By.CSS_SELECTOR, 'body > div:nth-child(5) > div.xl-section.clr > div.xl-section-r.fr > div > div.xl-table-view > div.xl-table-data > table > tbody')
    rows = table.find_elements(By.TAG_NAME, 'tr')
    for row in rows:
        # sales data
        cells = row.find_elements(By.TAG_NAME, 'td')
        line = [cell.text for cell in cells[:-1]]
        if line:
            model = line[1]
            sale = line[2]
            data.append([model, sale])
    
    print("Page %d Done!" % page)
    page += 1
    
    try:
        # wait for next page botton
        next_page = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'a.lineBlock.next')))
        # click next page
        next_page.click()
        # wait for next page
        wait.until(EC.staleness_of(next_page))
    except:
        print('All pages done!')
        break
driver.close()

Page 1 Done!
Page 2 Done!
Page 3 Done!
Page 4 Done!
Page 5 Done!
Page 6 Done!
Page 7 Done!
Page 8 Done!
Page 9 Done!
Page 10 Done!
Page 11 Done!
All pages done!


In [65]:
df = pd.DataFrame(data, columns=['model', 'sales'])
df['class'] = None
df['fuel'] = None
df['weight'] = None
df['frontB'] = None
df['rearB'] = None
df['link'] = None

## Get config

In [12]:
from tqdm import tqdm

In [113]:
def init_driver():
    service = Service(r'D:\OneDrive - 东南大学\5 我的代码\crawling\chromedriver\chromedriver.exe')
    chrome_options = Options()
    chrome_options.add_argument('--headless')
    chrome_options.add_experimental_option('excludeSwitches', ['enable-logging'])
    prefs = {"profile.managed_default_content_settings.images": 2, 'permissions.default.stylesheet': 2}
    chrome_options.add_experimental_option("prefs", prefs)
    chrome_options.add_argument('lang=zh_CN.utf-8')
    driver = webdriver.Chrome(service=service, options=chrome_options)
    return driver

### 1 extract links

In [88]:
url = 'https://auto.16888.com/'

service = Service(r'D:\OneDrive - 东南大学\5 我的代码\crawling\chromedriver\chromedriver.exe')
chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_experimental_option('excludeSwitches', ['enable-logging'])
prefs = {"profile.managed_default_content_settings.images": 2, 'permissions.default.stylesheet': 2}
chrome_options.add_experimental_option("prefs", prefs)
chrome_options.add_argument('lang=zh_CN.utf-8')
driver = webdriver.Chrome(service=service, options=chrome_options)

try:
    driver.get(url)
except:
    print("No Data.")
wait = WebDriverWait(driver, 10)

for i in tqdm(range(df.shape[0])):

    # get link
    line = df.loc[i]
    try:
        link = driver.find_element(By.LINK_TEXT, line['model'])
        link = link.get_attribute('href')
        df.loc[i, 'link'] = link
    except:
        pass
    
driver.close()

100%|██████████| 506/506 [07:48<00:00,  1.08it/s]


In [89]:
df.to_excel('./data/car_sale_2015_16.xlsx', index=False)

### 2 get configs

In [ ]:
for i in tqdm(range(df.shape[0])):
    
    link = df.loc[i, 'link']

    if link:
        driver = init_driver()
        driver.get(link)
    else:
        continue
    
    # select a year
    for y in list(range(2015, 2025, 1)):
        try:
            type_year = driver.find_element(By.LINK_TEXT, str(y)+'款')
            type_year.click()
            break
        except:
            pass
    driver.implicitly_wait(0.5)
    
    # get config data
    try:
        link = driver.find_element(By.LINK_TEXT, '参数配置')
        link.click()

        driver.implicitly_wait(0.5)
        config_table = driver.find_element(By.ID, 'config_main')  # data table
        config_data = config_table.text

        # config data extraction
        config_data = config_data.split('\n')
        car_class = config_data[5].split()[1:]
        fuel_type = config_data[49].split()[1:]
        weight = config_data[26].split()[1:]
        front_brake = config_data[66].split()[1:]
        rear_brake = config_data[67].split()[1:]

        # save
        df.loc[i, 'fuel'] = str(fuel_type)
        df.loc[i, 'class'] = str(car_class)
        df.loc[i, 'weight'] = str(weight)
        df.loc[i, 'frontB'] = str(front_brake)
        df.loc[i, 'rearB'] = str(rear_brake)
        
    except:
        pass

    driver.close()

In [139]:
df.to_excel('./data/car_sale_2015_16.xlsx', index=False)